# Set up notebook environment
## NOTE: Use a QIIME2 kernel

In [85]:
import os
import biom
import warnings
import pickle
import numpy as np
import pandas as pd
import qiime2 as q2
from biom import Table
from skbio import OrdinationResults
from skbio.stats import subsample_counts
from skbio.stats.distance import permanova, anosim, mantel
from skbio.stats.distance import DistanceMatrix
from qiime2.plugins.deicode.actions import rpca
from qiime2.plugins.feature_table.actions import rarefy
from qiime2.plugins.diversity.actions import beta_group_significance
from qiime2.plugins.emperor.actions import biplot, plot
from qiime2.plugins.diversity.actions import (beta,
                                              beta_phylogenetic,
                                              pcoa)
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition

from assets.step_wise_anova import run_stepwise_anova
from qiime2.plugins.fragment_insertion.actions import filter_features
warnings.filterwarnings("ignore", category=DeprecationWarning)

# helper functions
from assets.util import (mantel_matched, simulate_depth,
                        all_dists, nested_permanova)

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
%matplotlib inline


# Subset metadata to make paired files between extraction kits

In [86]:
SEQ_TYPE='shotgun'

In [87]:
# Read in sample metadata
md = pd.read_csv('Matrix_metadata_final1_HBM.tsv'.format(SEQ_TYPE=SEQ_TYPE), sep = '\t')
#md = md[(md['description'] != 'BLANK') & (md['description'] != 'POSITIVE_CONTROL')]
#md = md[~(md['sample_name'].isin([
#     '14332.363197859', '14332.361164111',
#     '14332.363197857', '14332.363197849',
#     '14332.363197875', 
#     '14332.363197831', '14332.363197893'

#]))]
#md = md[md.env_package == 'human-skin']
#md = md[md['env_feature'] == 'human-associated habitat']
#md.to_csv('shotgun/remove_blanks_md.tsv', sep='\t', index=False)
md

,sample_name,filename,ATTRIBUTE_storagebuffer,ATTRIBUTE_extraction_protocol,ATTRIBUTE_type,ATTRIBUTE_host_subject_id,ATTRIBUTE_sample_type2
0,363193206,363193206_GA11_01_64746.mzXML,isopropanol,Matrix,feces,mouse2,mouse_feces
1,363193214,363193214_GB10_01_64758.mzXML,isopropanol,Matrix,feces,mouse1,mouse_feces
2,363193215,363193215_GB11_01_64759.mzXML,isopropanol,Matrix,feces,mouse2,mouse_feces
3,363193225,363193225_GC10_01_64771.mzXML,isopropanol,Matrix,feces,mouse1,mouse_feces
4,363193232,363193232_GG11_01_64824.mzXML,isopropanol,Matrix,feces,mouse3,mouse_feces
5,363193235,363193235_GA12_01_64747.mzXML,isopropanol,Matrix,feces,mouse3,mouse_feces
6,363193242,363193242_GH11_01_64838.mzXML,isopropanol,Matrix,feces,mouse3,mouse_feces
7,363193260,363193260_GF12_01_64812.mzXML,isopropanol,Matrix,feces,mouse4,mouse_feces
8,363193261,363193261_GG12_01_64825.mzXML,isopropanol,Matrix,feces,mouse4,mouse_feces
9,363193268,363193268_GH10_01_64836.mzXML,isopropanol,Matrix,feces,mouse2,mouse_feces


# Stepwise RDA

## Shotgun data

In [88]:
# Import data
md_round1and2_bothPS_q2 = q2.Metadata.load('HBM_Matrix_LCMS_1500/metadata_ms_HBM.tsv')

table_shotgun = q2.Artifact.load('shotgun/tree-filtered-table.qza')

tree_shotgun = q2.Artifact.load('shotgun/tree.qza')


In [89]:
md_round1and2_bothPS_q2

Metadata
--------
84 IDs x 6 columns
filename:                      ColumnProperties(type='categorical')
ATTRIBUTE_storagebuffer:       ColumnProperties(type='categorical')
ATTRIBUTE_extraction_protocol: ColumnProperties(type='categorical')
ATTRIBUTE_type:                ColumnProperties(type='categorical')
ATTRIBUTE_host_subject_id:     ColumnProperties(type='categorical')
ATTRIBUTE_sample_type2:        ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation.

In [95]:
# Filter table
# table_shotgun_biom = table_shotgun.view(Table)
md_round1and2_bothPS_df = md_round1and2_bothPS_q2.to_dataframe()
# shared_ = list(set(table_shotgun_biom.ids()) & set(md_round1and2_bothPS_df.index))
md_round1and2_bothPS_df_shotgun = md_round1and2_bothPS_df#.reindex(shared_)
# table_shotgun_biom_bothPS = table_shotgun_biom.filter(shared_)
# keep_ = table_shotgun_biom_bothPS.ids('observation')[table_shotgun_biom_bothPS.sum('observation') > 0]
# table_shotgun_biom_bothPS.filter(keep_, axis='observation')

# # Import filtered table and re-indexed metadata file
# table_shotgun_bothPS = q2.Artifact.import_data('FeatureTable[Frequency]', table_shotgun_biom_bothPS)
md_round1and2_bothPS_df_shotgun_q2 = q2.Metadata(md_round1and2_bothPS_df_shotgun)

# # Generate distance matrices using 'all_dissts' utils
# rare_depth_shotgun = 1000
# dists_res_shotgun = all_dists(table_shotgun_bothPS,
#                       rare_depth_shotgun, tree_shotgun)

# Generate ordinations (row=samples, cols=axes)
pcoa_res_shotgun = {}
# pcoa_res_shotgun['Jaccard'] = pcoa(dists_res_shotgun['Jaccard'].distance_matrix).pcoa.view(OrdinationResults).samples
# pcoa_res_shotgun['Unweighted UniFrac'] = pcoa(dists_res_shotgun['Unweighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
# pcoa_res_shotgun['Weighted UniFrac'] = pcoa(dists_res_shotgun['Weighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
# pcoa_res_shotgun['RPCA'] = dists_res_shotgun['RPCA'].biplot.view(OrdinationResults).samples
distance_matrix = q2.Artifact.load('HBM_Matrix_LCMS_1500/distance_matrix1500_jaccard_qiime2_noise_detect2_noblanks_HBM_R.qza')
pcoa_res_shotgun['Jaccard'] = pcoa(distance_matrix).pcoa.view(OrdinationResults).samples
#names = {n: '14332.' + n for n in pcoa_res_shotgun['RPCA'].index} #appending "14332."to sample names in RPCA matrix
#pcoa_res_shotgun['RPCA'].rename(index=names, inplace=True)
#pcoa_res_shotgun["RPCA"] = pcoa_res_shotgun['RPCA'].loc[pcoa_res_shotgun['RPCA'].index.isin(pcoa_res_shotgun['Jaccard'].index)] #grabbing samples from the jaccard matrix only to use in RPCA #grabbing samples from the jaccard matrix only to use in RPCA

In [96]:
distance_matrix = q2.Artifact.load('HBM_Matrix_LCMS_1500/distance_matrix1500_jaccard_qiime2_noise_detect2_noblanks_HBM_R.qza')
distance_matrix

<artifact: DistanceMatrix uuid: e4865ce8-fe91-4ddf-95bc-1437d8c6c154>

In [97]:
# Perform stepwise RDA-ANOVA
es_all = {}
use_ = ['ATTRIBUTE_storagebuffer', 'ATTRIBUTE_host_subject_id', 'ATTRIBUTE_sample_type2']

# Clean up meta (only stuff to run)
mf_ord = md_round1and2_bothPS_df_shotgun_q2.to_dataframe().copy()
mf_ord = mf_ord[mf_ord.index.isin(pcoa_res_shotgun['Jaccard'].index)]

# Filter data
keep_ = [v_ for v_ in mf_ord.columns
         if len(set(mf_ord[v_])) > 1 and
         len(set(mf_ord[v_])) < mf_ord.shape[0]//2]
mf_ord = mf_ord[keep_]
print(len(keep_))
# Run stepwise ANOVA for all RDA ordinations
for metric_, ord_ in  pcoa_res_shotgun.items():
    # get first three axes
    ord_ = ord_[[0,1,2]]
    ord_.columns = ['PC1','PC2','PC3']
    # subset/match
    mf_ord_ = mf_ord.copy()
    shared_ids = list(set(ord_.index)\
                      & set(mf_ord_.index))
    mf_ord_ = mf_ord_.loc[shared_ids,:]
    ord_ = ord_.loc[shared_ids,:]
    es_all[metric_] = run_stepwise_anova(ord_, mf_ord_, use_) #mf_ord_.columns)

# Concat output from all runs and export
es_alldf = pd.concat(es_all).rename({'+ sample_type2':'+ sample_type'}, axis=0)
# es_alldf.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/stepwise_anova/stepwise_anova_shotgun.txt', sep='\t')
es_alldf


4
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: /Users/caitrionabrennan/Downloads/assets/stepwise-rda.R /var/folders/lh/d1kdwkz53856l7_mtjyz0f640000gn/T/tmpbyx3p8ln/ord_.tsv /var/folders/lh/d1kdwkz53856l7_mtjyz0f640000gn/T/tmpbyx3p8ln/mf_.txt /var/folders/lh/d1kdwkz53856l7_mtjyz0f640000gn/T/tmpbyx3p8ln/output.effect.size.tsv

R version 4.0.5 (2021-03-31) 


Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ ATTRIBUTE_storagebuffer +
ATTRIBUTE_host_subject_id + ATTRIBUTE_sample_type2, data = X_16S, scale
= TRUE)

              Inertia Proportion Rank
Total          3.0000     1.0000     
Constrained    1.4815     0.4938    3
Unconstrained  1.5185     0.5062    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.8956 0.4417 0.1442 

Eigenvalues for unconstrained axes:
   PC1    PC2    PC3 
0.8558 0.5583 0.1044 



R2.adj  Df        AIC          F  \
Jaccard + ATTRIBUTE_sample_type2     0.370230   3  57.343342  17.264750   
        + ATTRIBUTE_host_subject_id  0.053166   6  55.385923   2.229394   

                                       Pr(>F)  
Jaccard + ATTRIBUTE_sample_type2     0.000200  
        + ATTRIBUTE_host_subject_id  0.012198

In [43]:
pcoa_res_shotgun["RPCA"].shape,pcoa_res_shotgun["Jaccard"].shape

KeyError: 'Jaccard'

In [44]:
mf_ord.columns

Index(['ATTRIBUTE_storagebuffer', 'ATTRIBUTE_host_subject_id',
       'ATTRIBUTE_sample_type2'],
      dtype='object')

In [45]:
s1=set(pcoa_res_shotgun['RPCA'].index)

In [46]:
s2=set(pcoa_res_shotgun['Jaccard'].index)

KeyError: 'Jaccard'

In [47]:
s1

{'361162769',
 '361162770',
 '361162771',
 '361162789',
 '361162790',
 '361162793',
 '361162798',
 '361162805',
 '361162807',
 '361162809',
 '361162812',
 '361162817',
 '361164071',
 '361164072',
 '361164081',
 '361164082',
 '361164083',
 '361164086',
 '361164090',
 '361164091',
 '361164093',
 '361164094',
 '361164109',
 '361164117',
 '361164167',
 '361164168',
 '361164169',
 '361164170',
 '361164173',
 '361164174',
 '361164175',
 '361164176',
 '361164178',
 '361164180',
 '361164183',
 '361164184',
 '361164186',
 '361164187',
 '361164188',
 '361164190',
 '361164195',
 '361164196',
 '361164197',
 '361164198',
 '361164199',
 '361164201',
 '361164203',
 '361164213',
 '361164223',
 '361164224',
 '361164229',
 '361164231',
 '361164234',
 '361164244',
 '361164250',
 '361164254',
 '361164255',
 '361164259',
 '361164260',
 '363193206',
 '363193214',
 '363193215',
 '363193225',
 '363193232',
 '363193235',
 '363193242',
 '363193260',
 '363193261',
 '363193268',
 '363193269',
 '363193276',
 '3631